In [2]:
import requests
import json
import io
import os
import csv
import pandas as pd
import openpyxl

# Getting the Access Token

In [4]:
header_url = 'https://counter-club.us.hivebrite.com/api'

oauth_url = '/oauth/token'

payload = {
    'grant_type': "password",
    "admin_email": "abbie@counterpart.vc",
    "password": "SanFran2023!",
    "client_id": "OXq_LdzDg81ZP-1tDPKmvUBhhGKO02_QaJO3xZXYLL8",
    "client_secret": "Wl0TmLIu-XskM5XHIdjIfzaZfwcCumVzeoTiaDvL0dQ",
    "scope": "admin"
}
response = requests.post(f'{header_url}{oauth_url}', data=payload)

access_token = response.json().get('access_token', None)

print(access_token)

QXjbyhfhqMjNcf4KE1ylLbEyvV2kJR4-YfCnOaA_sTQ


# Importing Industries

## Getting Industries from Hivebrite API

In [5]:
industry_url = "/admin/v1/settings/industries"
headers = {
    'Authorization': f'Bearer {access_token}'
}

industries = requests.get(f'{header_url}{industry_url}', headers=headers)
industries = industries.json()['industries']

## Data Manipulation

Combining Company ID with Industry ID

In [6]:
# the file we're pulling data from
file_path = "/Users/counterpartventures/Desktop/Data/affinity/company_upload.xlsx"
df = pd.read_excel(file_path, sheet_name="industry_only")
industry_code = pd.DataFrame(industries)

df.columns = ["company_id", "name"]

# Split the industry by commas
df = df.assign(name=df['name'].str.split(', ')).explode('name')
df = df.merge(industry_code, on="name", how="left").dropna()
df['id'] = df['id'].astype(int)

# Group by 'company_id' and summarize
df = df.groupby('company_id').agg({'id': list}).reset_index()

# Convert to JSON and save
json_df = df.to_json(orient='records', indent=2)

# load to use in Py
json_obj = json.loads(json_df)

json_obj[326]

{'company_id': 1339485, 'id': [1748, 13527]}

### Testing insert

In [196]:
test_url = json_obj[326]['company_id']
test_id = json_obj[326]['id']

test_data = {
    'company[industries][][id]' : test_id
}

requests.put(f'{header_url}{company_header}{test_url}', headers = headers, data = test_data)

<Response [200]>

## Putting Industries into Affinity

In [160]:
for entry in json_obj:
    temp_url = entry['company_id']
    temp_id = entry['id']

    temp_data = {
        'company[industries][][id]' : temp_id
    }

    requests.put(f'{header_url}{company_header}{temp_url}', headers = headers, data = temp_data)

# Adding Long Description

In [7]:
descrip = pd.read_excel(file_path, sheet_name="long_description")

descrip_json = descrip.to_json(orient='records', indent=1)

descrip_obj = json.loads(descrip_json)b

## Testing Insert

In [314]:
test_url = descrip_obj[365]['ID']
test_descrip = descrip_obj[365]['long_description']

test_data = {
    'company[long_description]' : test_descrip
}

requests.put(f'{header_url}{company_header}{test_url}', headers = headers, data = test_data)

<Response [200]>

In [315]:
for entry in descrip_obj:
    temp_url = entry['ID']
    temp_descrip = entry['long_description']

    temp_data = {
        'company[long_description]' : temp_descrip
    }

    requests.put(f'{header_url}{company_header}{temp_url}', headers = headers, data = temp_data)